# Factual Memory: Personalized Communication Styles

This notebook demonstrates how **Factual Memory** enables AI agents to adapt their communication style, technical level, and approach based on stored customer preferences and facts. Unlike episodic memory which recalls events, factual memory stores enduring customer attributes like communication preferences, risk tolerance, and decision-making styles.

## Dependencies

Core libraries for building adaptive communication systems that personalize responses based on customer preferences and factual profiles.

In [2]:
import asyncio
from typing import Dict, List, Any, TypedDict
from datetime import datetime
import json
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_aws import ChatBedrockConverse
from mem0 import MemoryClient
import os

## LLM Configuration


In [ ]:
llm = ChatBedrockConverse(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    temperature=0.1,
)

## Memory Client Setup

Initializes the memory client for storing and retrieving customer factual profiles including communication preferences, technical level, and decision-making styles.

In [ ]:
import os
import getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("MEM0_API_KEY")

In [ ]:
import os
from mem0 import MemoryClient

memory_client = MemoryClient()

## Factual Memory State

Defines the workflow state for processing queries with customer factual profiles and preferences to enable personalized communication.

In [4]:
class FactualMemoryState(TypedDict):
    """Simple state for factual memory"""
    customer_id: str
    customer_query: str
    factual_memories: List[Dict[str, Any]]
    response: str

## Factual Memory Retrieval

Searches for customer preferences, communication styles, and factual attributes to inform personalized response generation.

In [5]:
async def retrieve_factual_memory(state: FactualMemoryState) -> FactualMemoryState:
    """Retrieve customer's factual information"""
    
    # Get all factual information about this customer
    factual_memories = memory_client.search(
        query="preferences communication style contact method risk tolerance business context",
        user_id=state["customer_id"],
        limit=10
    )
    
    state["factual_memories"] = factual_memories
    return state

## Response Generation

Adapts communication style, technical level, and format based on customer preferences to create personalized responses.

In [6]:
async def generate_factual_aware_response(state: FactualMemoryState) -> FactualMemoryState:
    """Generate response tailored to customer's factual profile"""
    
    # Extract factual information
    factual_context = "\n".join([
        memory.get('memory', '') for memory in state["factual_memories"]
    ])
    
    prompt = f"""
You are an insurance support agent. Respond according to the customer's known preferences and facts.

Customer Query: {state["customer_query"]}

Customer's Known Preferences and Facts:
{factual_context if factual_context else "No preferences found - use standard professional tone"}

Instructions:
- Adapt your communication style based on their stated preferences
- Use appropriate technical level (simple for non-technical, detailed for technical)
- Respect their contact preferences (email/phone/text)
- Consider their risk tolerance (conservative/moderate/aggressive)
- Account for their personal context (retired/busy/business owner)
- Match their decision-making style (analytical/emotional/practical)

Generate a response that feels personally tailored to them.
"""

    response = await llm.ainvoke(prompt)
    state["response"] = response.content.strip()
    
    return state



## Memory Updates

Stores the current interaction to continuously refine the customer's factual profile and preferences.

In [24]:
async def update_factual_memory(state: FactualMemoryState) -> FactualMemoryState:
    
    memory_client.add(
        messages=[{"role": "user", "content": state['customer_query']},{"role": "assistant", "content": state['response']}],
        user_id=state['customer_id'],
        metadata={
            "type": "factual_memory",
           
        }
    )
    
    return state

## Factual Memory Workflow

Creates a workflow that retrieves customer preferences, generates personalized responses, and updates factual profiles.

In [9]:
def create_factual_workflow() -> StateGraph:
    """Create workflow for factual memory demo"""
    
    workflow = StateGraph(FactualMemoryState)
    
    # Add nodes
    workflow.add_node("retrieve_factual_memory", retrieve_factual_memory)
    workflow.add_node("generate_factual_aware_response", generate_factual_aware_response)
    workflow.add_node("update_factual_memory", update_factual_memory)
    
    # Define flow
    workflow.set_entry_point("retrieve_factual_memory")
    workflow.add_edge("retrieve_factual_memory", "generate_factual_aware_response")
    workflow.add_edge("generate_factual_aware_response", "update_factual_memory")
    workflow.add_edge("update_factual_memory", END)
    
    # Compile
    memory_saver = MemorySaver()
    return workflow.compile(checkpointer=memory_saver)

## Workflow Compilation

Compiles the factual memory workflow for processing personalized customer interactions.

In [25]:
graph = create_factual_workflow()

## Sample conversations

Creates diverse customer profiles with different communication preferences: a retired customer who prefers simple explanations and phone calls, a tech-savvy business owner who wants detailed data via email, and a busy parent who needs brief text messages.

In [19]:
def store_customer_factual_profiles():
    """Store diverse customer factual profiles"""
    
    # Customer 1: Retired, non-technical, conservative
    retired_customer = {
        "conversation_id": "FACT-001",
        "customer_id": "CUST-67890",
        "timestamp": "2024-01-20T10:00:00Z",
        "conversation": [
            {
                "role": "user",
                "content": "Hi, I need to review my policies. I'm not very tech-savvy, so please keep explanations simple. Also, I prefer phone calls over emails - I can never find emails in my inbox!"
            },
            {
                "role": "assistant",
                "content": "Absolutely! I'll keep everything straightforward and avoid technical jargon. I've noted that you prefer phone communication. Would you like me to call you to discuss your policy review?"
            },
            {
                "role": "user",
                "content": "Yes, please. I'm usually available mornings before 11 AM. I'm retired so I have time to talk through things properly."
            },
            {
                "role": "assistant",
                "content": "Perfect! I've noted that morning calls work best for you. Since you have time for detailed discussions, I'll make sure we cover everything thoroughly but in simple terms."
            },
            {
                "role": "user",
                "content": "All of them - home, auto, and life. I'm very safety-conscious, prefer comprehensive coverage even if it costs more. Peace of mind is worth it at my age."
            }
        ]
    }
    
    # Store the conversation
    memory_client.add(
        messages=retired_customer["conversation"],
        user_id=retired_customer["customer_id"]
    )
    
    # Customer 2: Tech-savvy business owner
    business_owner = [
        {
            "content": "I run a software consulting firm with 20 employees. Need everything documented for our records.",
 
        },
        {
            "content": "Email me all details - I prefer written communication with data and analytics. I'm very comfortable with technical information.",
        },
        {
            "content": "I make decisions based on ROI analysis. Cost-efficiency is important but I understand the value of proper coverage.",
        },
        {
            "content": "I typically review documents in the evenings after 6 PM. Quick email responses work best for my schedule.",

        }
    ]
    
    for fact in business_owner:
        memory_client.add(
            messages=[{"role": "user", "content": fact["content"]}],
            user_id="CUST-TECH-BIZ-001"
        )
    
    # Customer 3: Busy parent
    busy_parent = [
        {
            "content": "I have three young kids and work full-time as a nurse. Text me - I can't take calls during my shifts.",
            "metadata": {"type": "factual_memory"}
        },
        {
            "content": "Keep things brief and to the point. I don't have time for long explanations. Just tell me what I need to know.",
            "metadata": {"type": "factual_memory"}
        },
        {
            "content": "My spouse and I make insurance decisions together. We're moderate with risk - want good coverage but need to watch the budget.",
            "metadata": {"type": "factual_memory"}
        },
        {
            "content": "Best times to reach me are early morning (6-7 AM) or late evening (after 8 PM) when kids are asleep.",
            "metadata": {"type": "factual_memory"}
        }
    ]
    
    for fact in busy_parent:
        memory_client.add(
            messages=[{"role": "user", "content": fact["content"]}],
            user_id="CUST-PARENT-001",
        )

## Loading Customer Profiles

Executes the storage of diverse customer factual profiles into memory for demonstration purposes.

In [20]:
# Store customer profiles
print("📚 [Loading customer factual profiles...]")
store_customer_factual_profiles()
print("✅ Customer profiles loaded!\n")


📚 [Loading customer factual profiles...]
✅ Customer profiles loaded!



## Query Handler

Processes customer queries through the factual memory workflow to generate responses tailored to individual communication preferences and styles.

In [21]:
async def handle_query_with_factual_memory(customer_id: str, query: str) -> Dict[str, Any]:
    """Process a query using factual memory"""
    
    initial_state = FactualMemoryState(
        customer_id=customer_id,
        customer_query=query,
        factual_memories=[],
        response=""
    )
    
    config = {"configurable": {"thread_id": f"factual_{customer_id}"}}
    result = await graph.ainvoke(initial_state, config)
    
    return {
        "query": query,
        "response": result["response"],
        "facts_found": len(result["factual_memories"])
    }

## Factual Memory Demonstration

The following examples show how the same insurance query generates vastly different responses based on each customer's factual profile - adapting communication style, technical level, format, and timing preferences.

In [26]:

# SCENARIO 1: Rate increase notification - different customers
print("💰 SCENARIO 1: Auto Insurance Rate Increase")
print("=" * 80)

rate_query = "My auto insurance went up by $50 this month. Why?"

# Retired customer
print("\n👴 Retired Customer (Non-technical, Phone Preference):")
print("-" * 60)

print("\n\n✅ WITH FACTUAL MEMORY:")
retired_result = await handle_query_with_factual_memory("CUST-67890", rate_query)
print(f"Customer: {rate_query}")
print(f"Agent: {retired_result['response']}")
print(f"💾 Facts used: {retired_result['facts_found']}")

# %%


💰 SCENARIO 1: Auto Insurance Rate Increase

👴 Retired Customer (Non-technical, Phone Preference):
------------------------------------------------------------


✅ WITH FACTUAL MEMORY:
Customer: My auto insurance went up by $50 this month. Why?
Agent: Based on the customer's preferences, here's a tailored response:

Good morning! I noticed you might have some questions about your recent auto insurance rate increase. I'd be happy to give you a call tomorrow morning before 11 AM to walk you through the details in a way that's easy to understand. 

From what I know about your preference for comprehensive coverage and safety, there could be a few reasons for the rate adjustment:
- Recent driving record updates
- Changes in local accident or claim rates
- Vehicle age or value adjustments
- Comprehensive coverage enhancements

Would you like me to prepare a detailed breakdown to review during our phone call? I want to ensure you have complete peace of mind about your policy and understand exa

In [27]:
# Business owner
print("\n\n💼 Business Owner (Technical, Email Preference):")
print("-" * 60)

business_result = await handle_query_with_factual_memory("CUST-TECH-BIZ-001", rate_query)
print(f"Customer: {rate_query}")
print(f"Agent: {business_result['response']}")
print(f"💾 Facts used: {business_result['facts_found']}")





💼 Business Owner (Technical, Email Preference):
------------------------------------------------------------
Customer: My auto insurance went up by $50 this month. Why?
Agent: Subject: Auto Insurance Premium Adjustment - Detailed Analysis

Dear [Customer],

Per your preference for data-driven communication, I'm providing a comprehensive breakdown of your recent $50 premium increase.

Key Factors Contributing to Rate Adjustment:
1. Risk Pool Recalibration
- Regional claims data indicates increased accident frequency
- Statistical model suggests 7.2% overall risk increment in your zip code

2. Vehicle Risk Assessment
- Your current vehicle model shows slightly higher repair costs
- Insurance risk algorithms reflect emerging repair technology expenses

3. Actuarial Model Updates
- Recent predictive analytics suggest adjusted risk profiles
- Micro-segmentation algorithms triggered premium recalibration

Recommended Actions:
- Review current coverage alignment with business vehicle usage
